In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
import sv_ttk #dark ttk theme

import pandas as pd
import matplotlib.pyplot as plt
import os

# Useful helper functions

In [2]:
#chatgpt generated
def import_file(filename):
    """
    Imports a file and stores its data in a structured format.
    Supports CSV, Excel, JSON, and text files.
    Returns a Pandas DataFrame if possible; otherwise, returns raw text or structured data.
    """
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File '{filename}' not found.")

    file_extension = os.path.splitext(filename)[-1].lower()

    try:
        if file_extension in ['.csv']:
            return pd.read_csv(filename)
        elif file_extension in ['.xlsx', '.xls']:
            return pd.read_excel(filename)
        elif file_extension in ['.json']:
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
                return pd.json_normalize(data) if isinstance(data, list) else data
        elif file_extension in ['.txt']:
            with open(filename, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            raise ValueError(f"Unsupported file format: {file_extension}")
    except Exception as e:
        raise RuntimeError(f"Error processing file '{filename}': {str(e)}")

# Main app

In [48]:
#the image part
class DataVisualizer(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent

#basic analysis and summary of imported data
class DataPreview(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent

#all the buttons to make stuff happen
class SelectionPanel(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent
        self.columnconfigure(0, weight=1)
        self.rowconfigure(0, weight=1)
        self.rowconfigure(1, weight=1)
        
        self.visualizaion_mode_dropdown = ttk.Combobox(
            state="readonly",
            values=["Graph", "Correlation Matrix"]
        )
        self.visualizaion_mode_dropdown.grid(row=0, column=0, sticky='nsew')

    def update_visibility(self, loaded_data):
        pass
            

#main app structure and functions
class App(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent
        
        self.import_btn = ttk.Button(text="Import data", command=self.select_file)
        self.status_text = ttk.Label(text="")
        self.selection_panel = SelectionPanel(self)
        self.data_preview = DataPreview(self)
        self.data_visualizer = DataVisualizer(self)

        self.import_btn.grid(row=0, column=0)
        self.status_text.grid(row=2, column=0)
        self.selection_panel.grid(row=0, column=1, sticky='nsew')
        self.data_preview.grid(row=1, column=0)
        self.data_visualizer.grid(row=0, column=2)

        self.loaded_file = None

    def select_file(self):
        """
        Uses tkinter to open a file exploration panel to allow the user to select a file.
        Uses the import_file function to load the file into a PD object.
        Runs some basic data cleaning on loaded data
        """
        filename = askopenfilename() # get file location using tkinter
        try:
            self.loaded_data = import_file(filename)
            self.loaded_data.fillna(0)
            self.status_text.config(text='Data loaded!', foreground='green')
            self.selection_panel.update_visibility(self.loaded_data)
        except Exception as e:
            print(e)
            self.status_text.config(text="Can't open file", foreground='red')

root = tk.Tk()
root.title('Data analysis program')
root.geometry("1020x500")

root.columnconfigure(0, weight=1)
root.columnconfigure(1, weight=10)
root.columnconfigure(2, weight=10)
root.rowconfigure(0, weight=1)
root.rowconfigure(1, weight=10)
root.rowconfigure(2, weight=3)

app = App(root)
sv_ttk.set_theme("dark") #make the app evil (dark theme)
root.mainloop()

In [11]:
df.corr().style.background_gradient(cmap='coolwarm')

,quantity,unit_price,customer_satisfaction,days_since_last_purchase,was_returned,account_age_days,total_price
quantity,1.000000,-0.010960,-0.003107,0.009596,0.001175,0.012220,0.239623
unit_price,-0.010960,1.000000,-0.065573,-0.004682,0.025872,0.011316,0.892345
customer_satisfaction,-0.003107,-0.065573,1.000000,0.008204,-0.386844,0.008388,-0.063548
days_since_last_purchase,0.009596,-0.004682,0.008204,1.000000,-0.013499,-0.003705,0.002475
was_returned,0.001175,0.025872,-0.386844,-0.013499,1.000000,-0.006163,0.023394
account_age_days,0.012220,0.011316,0.008388,-0.003705,-0.006163,1.000000,0.014721
total_price,0.239623,0.892345,-0.063548,0.002475,0.023394,0.014721,1.000000
